# tensorboard

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# summary 값을 logs폴더에 저장
log_path = "./logs/xor"
writer = tf.summary.create_file_writer(log_path)

In [3]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

# float값으로 바꾸기 위해
def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

#w1 = tf.Variable(tf.random.normal([2,1]), name='weight1')
#b1 = tf.Variable(tf.random.normal([1], name = 'bias1'))

#w2 = tf.Variable(tf.random.normal([2,1]), name='weight2')
#b2 = tf.Variable(tf.random.normal([1], name = 'bias2'))

#w3 = tf.Variable(tf.random.normal([2,1]), name='weight3')
#b3 = tf.Variable(tf.random.normal([1], name = 'bias3'))

#w4 = tf.Variable(tf.random.normal([2,1]), name='weight4')
#b4 = tf.Variable(tf.random.normal([1], name = 'bias4'))

w1 = tf.Variable(tf.random.normal((2, 10)), name='weight1')
b1 = tf.Variable(tf.random.normal((10,)), name='bias1')

w2 = tf.Variable(tf.random.normal((10, 10)), name='weight2')
b2 = tf.Variable(tf.random.normal((10,)), name='bias2')

w3 = tf.Variable(tf.random.normal((10, 10)), name='weight3')
b3 = tf.Variable(tf.random.normal((10,)), name='bias3')

w4 = tf.Variable(tf.random.normal((10, 1)), name='weight4')
b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

In [4]:
def neural_net(features, step):
    layer1 = tf.sigmoid(tf.matmul(features, w1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, w3) + b3)
    hypothesis = tf.sigmoid(tf.matmul(layer3, w4) + b4)
    with writer.as_default():
        tf.summary.histogram("weight1", w1, step=step)
        tf.summary.histogram("bias1", b1, step=step)
        tf.summary.histogram("layer1", layer1, step=step)

        tf.summary.histogram("weight2", w2, step=step)
        tf.summary.histogram("bias2", b2, step=step)
        tf.summary.histogram("layer2", layer2, step=step)

        tf.summary.histogram("weight3", w3, step=step)
        tf.summary.histogram("bias3", b3, step=step)
        tf.summary.histogram("layer3", layer3, step=step)

        tf.summary.histogram("weight4", w4, step=step)
        tf.summary.histogram("bias4", b4, step=step)
        tf.summary.histogram("hypothesis", hypothesis, step=step)

    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1-labels) * tf.math.log(1 - hypothesis))
    with writer.as_default():
        tf.summary.scalar('loss', cost, step=step)
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

def accuracy_fn(hypothesis,labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(features, labels,step):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features,step), labels)
    return tape.gradient(loss_value, [w1,w2,w3,w4,b1,b2,b3,b4])

EPOCHS = 3000
for step in range(EPOCHS):
    for features, labels in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(features, labels, step)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [w1,w2,w3,w4,b1,b2,b3,b4]))
        if step % 50 == 0:
            loss_value = loss_fn(neural_net(features,step),labels)
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))

x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data,step), y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))


Iter: 0, Loss: 1.0230
Iter: 50, Loss: 0.8590
Iter: 100, Loss: 0.7652
Iter: 150, Loss: 0.7202
Iter: 200, Loss: 0.7009
Iter: 250, Loss: 0.6931
Iter: 300, Loss: 0.6900
Iter: 350, Loss: 0.6886
Iter: 400, Loss: 0.6880
Iter: 450, Loss: 0.6878
Iter: 500, Loss: 0.6876
Iter: 550, Loss: 0.6873
Iter: 600, Loss: 0.6871
Iter: 650, Loss: 0.6870
Iter: 700, Loss: 0.6868
Iter: 750, Loss: 0.6867
Iter: 800, Loss: 0.6864
Iter: 850, Loss: 0.6862
Iter: 900, Loss: 0.6862
Iter: 950, Loss: 0.6859
Iter: 1000, Loss: 0.6858
Iter: 1050, Loss: 0.6855
Iter: 1100, Loss: 0.6853
Iter: 1150, Loss: 0.6852
Iter: 1200, Loss: 0.6850
Iter: 1250, Loss: 0.6848
Iter: 1300, Loss: 0.6847
Iter: 1350, Loss: 0.6845
Iter: 1400, Loss: 0.6843
Iter: 1450, Loss: 0.6842
Iter: 1500, Loss: 0.6840
Iter: 1550, Loss: 0.6838
Iter: 1600, Loss: 0.6835
Iter: 1650, Loss: 0.6834
Iter: 1700, Loss: 0.6831
Iter: 1750, Loss: 0.6829
Iter: 1800, Loss: 0.6828
Iter: 1850, Loss: 0.6825
Iter: 1900, Loss: 0.6824
Iter: 1950, Loss: 0.6822
Iter: 2000, Loss: 0.682

In [5]:
# summary 값을 logs폴더에 저장
log_path = "./logs/xor"
writer = tf.summary.create_file_writer(log_path)

In [6]:
%load_ext tensorboard

In [7]:
%tensorboard --logdir logs/xor # 왜 데이터셋이 없다고 뜨지..